In [1]:
from skimage import feature
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.morphology import erosion, dilation, rectangle, square
from skimage.measure import find_contours

from scipy import ndimage as ndi
from scipy.misc import imsave

%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

from collections import defaultdict
from copy import deepcopy

#import pyscreenshot as pysc
from PIL import ImageGrab
# from PIL import Image

import random
import math
from time import sleep

In [2]:
mystery = ndi.imread('grid_square_mystery.png', mode='L')
empty = ndi.imread('grid_square_empty.png', mode='L')
grid_one = ndi.imread('grid_one.png', mode='L')
grid_two = ndi.imread('grid_two.png', mode='L')
grid_three = ndi.imread('grid_three.png', mode='L')
minesweep = ndi.imread('my_shot.png', mode='L')
smiley_face = ndi.imread('smiley_face.png', mode='L')
flag = ndi.imread('flag.png', mode='L')
#ndi.imshow(flag)


In [19]:
from Quartz.CoreGraphics import CGEventCreateMouseEvent
from Quartz.CoreGraphics import CGEventPost
from Quartz.CoreGraphics import kCGEventMouseMoved
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseUp
from Quartz.CoreGraphics import kCGMouseButtonLeft
from Quartz.CoreGraphics import kCGHIDEventTap


def mouseEvent(type, posx, posy):
        theEvent = CGEventCreateMouseEvent(
                    None, 
                    type, 
                    (posx,posy), 
                    kCGMouseButtonLeft)
        CGEventPost(kCGHIDEventTap, theEvent)

def mousemove(posx,posy):
        mouseEvent(kCGEventMouseMoved, posx,posy);

def mouseclick(posx,posy):
        # uncomment this line if you want to force the mouse 
        # to MOVE to the click location first (I found it was not necessary).
        #mouseEvent(kCGEventMouseMoved, posx,posy);
        mouseEvent(kCGEventLeftMouseDown, posx,posy);
        mouseEvent(kCGEventLeftMouseUp, posx,posy);

In [20]:
fullscreen = ImageGrab.grab().convert('L')  # X1,Y1,X2,Y2
#fullscreen.show()
fs = np.array(fullscreen)

In [21]:
simil_mat = cv2.matchTemplate(fs, smiley_face, cv2.TM_CCOEFF_NORMED)
thing = cv2.minMaxLoc(simil_mat)

In [22]:
max_loc = thing[3]

In [24]:
mouseclick(max_loc[0]/2 - 10, max_loc[1]/2)
mines = ImageGrab.grab(bbox = (max_loc[0]-900, max_loc[1]-85, (max_loc[0]) + 955, (max_loc[1]) + 1100)).convert('L')  # X1,Y1,X2,Y2
mines.show()
mines = np.array(mines)

In [15]:
mysteries = cv2.matchTemplate(mines, mystery, cv2.TM_CCOEFF_NORMED)


In [16]:
def dist((x1,y1),(x2,y2)):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [17]:
def remove_close_points(spots, similarity_matrix, threshold = 5):
    spotscopy = deepcopy(spots)
    for i in range(len(spotscopy)):
        for j in range(len(spotscopy)):
            a = spotscopy[i]
            b = spotscopy[j]
            if dist(a,b) < threshold and j > i:
                if similarity_matrix[a[0]][a[1]] > similarity_matrix[b[0]][b[1]] and b in spots:
                    spots.remove(b)
                elif similarity_matrix[a[0]][a[1]] < similarity_matrix[b[0]][b[1]] and a in spots:
                    spots.remove(a)
    return spots

In [18]:
def find_candidate_locations(similarity_matrix):
    candidate_spots = []
    thing = np.where(similarity_matrix > .8)
#     for n in range(len(similarity_matrix)):
#         for m in range(len(similarity_matrix[n])):
#             if similarity_matrix[n][m] > .95:
#                 candidate_spots.append((n,m))
    return zip(thing[0], thing[1])
#     final_spots = remove_close_points(candidate_spots, similarity_matrix)
#     return final_spots

In [19]:
grid = np.negative(np.ones((9,9)))

In [20]:
grid_locations = remove_close_points(find_candidate_locations(mysteries), mysteries)

In [21]:
ones = cv2.matchTemplate(mines, grid_one, cv2.TM_CCOEFF_NORMED)
flag_locations = remove_close_points(find_candidate_locations(ones), ones)

In [22]:
flag_locations

[]

In [83]:
grid_locations

[(85, 490),
 (195, 160),
 (195, 220),
 (195, 280),
 (195, 340),
 (195, 400),
 (195, 460),
 (195, 520),
 (195, 580),
 (195, 640),
 (195, 700),
 (195, 760),
 (195, 820),
 (195, 880),
 (195, 940),
 (255, 160),
 (255, 220),
 (255, 280),
 (255, 340),
 (255, 400),
 (255, 460),
 (255, 520),
 (255, 580),
 (255, 640),
 (255, 700),
 (255, 760),
 (255, 820),
 (255, 880),
 (255, 940),
 (315, 40),
 (315, 100),
 (315, 160),
 (315, 220),
 (315, 280),
 (315, 340),
 (315, 400),
 (315, 460),
 (315, 520),
 (315, 580),
 (315, 640),
 (315, 700),
 (315, 760),
 (315, 820),
 (315, 880),
 (315, 940),
 (375, 40),
 (375, 100),
 (375, 160),
 (375, 220),
 (375, 280),
 (375, 340),
 (375, 400),
 (375, 460),
 (375, 520),
 (375, 580),
 (375, 640),
 (375, 700),
 (375, 760),
 (375, 820),
 (375, 880),
 (375, 940),
 (435, 220),
 (435, 280),
 (435, 340),
 (435, 400),
 (435, 460),
 (435, 520),
 (435, 580),
 (435, 640),
 (435, 700),
 (435, 760),
 (435, 820),
 (435, 880),
 (435, 940),
 (495, 220),
 (495, 340),
 (495, 580),
 (

In [80]:
flag_locations

[(195, 40),
 (195, 100),
 (255, 40),
 (255, 100),
 (495, 280),
 (1035, 40),
 (1035, 100),
 (1095, 40),
 (1095, 100)]

In [56]:
empty_locations

[(57, 526),
 (483, 22),
 (483, 82),
 (543, 22),
 (543, 82),
 (543, 442),
 (603, 22),
 (603, 82),
 (603, 142),
 (603, 202),
 (603, 262),
 (603, 322),
 (603, 382),
 (663, 22),
 (663, 82),
 (721, 20),
 (721, 80),
 (781, 20),
 (781, 80),
 (841, 140),
 (843, 20),
 (843, 80),
 (903, 140)]

In [51]:
three_locations # -12, -18

[(483, 502)]

In [43]:
one_locations # -12, -18

[(423, 22),
 (423, 82),
 (423, 142),
 (483, 142),
 (483, 382),
 (483, 442),
 (543, 142),
 (543, 202),
 (543, 322),
 (543, 382),
 (543, 502),
 (603, 442),
 (603, 502),
 (663, 142),
 (663, 202),
 (663, 262),
 (663, 322),
 (663, 382),
 (723, 142),
 (783, 142),
 (783, 202),
 (843, 202),
 (903, 22),
 (903, 82),
 (903, 202),
 (963, 142)]

In [22]:
grid_locations

[(195, 40),
 (195, 100),
 (195, 160),
 (195, 220),
 (195, 280),
 (195, 340),
 (195, 400),
 (195, 460),
 (195, 520),
 (195, 580),
 (195, 640),
 (195, 700),
 (195, 760),
 (195, 820),
 (195, 880),
 (195, 940),
 (255, 40),
 (255, 100),
 (255, 160),
 (255, 220),
 (255, 280),
 (255, 340),
 (255, 400),
 (255, 460),
 (255, 520),
 (255, 580),
 (255, 640),
 (255, 700),
 (255, 760),
 (255, 820),
 (255, 880),
 (255, 940),
 (315, 40),
 (315, 100),
 (315, 160),
 (315, 220),
 (315, 280),
 (315, 340),
 (315, 400),
 (315, 460),
 (315, 520),
 (315, 580),
 (315, 640),
 (315, 700),
 (315, 760),
 (315, 820),
 (315, 880),
 (315, 940),
 (375, 40),
 (375, 100),
 (375, 160),
 (375, 220),
 (375, 280),
 (375, 340),
 (375, 400),
 (375, 460),
 (375, 520),
 (375, 580),
 (375, 640),
 (375, 700),
 (375, 760),
 (375, 820),
 (375, 880),
 (375, 940),
 (435, 40),
 (435, 100),
 (435, 160),
 (435, 220),
 (435, 280),
 (435, 340),
 (435, 400),
 (435, 460),
 (435, 520),
 (435, 580),
 (435, 640),
 (435, 700),
 (435, 760),
 (43

In [23]:
grid_loc_dict = dict()
for i in range(len(grid_locations)):
    tup = grid_locations[i]
    grid_loc_dict[tup] = (i/9, i % 9)

grid_coord_dict = dict()

for x in grid_loc_dict.items():
    grid_coord_dict[x[1]] = x[0]

In [121]:
mouseclick((max_loc[0]-275)/2 + 17 + 15 , (max_loc[1]-85)/2 + 100 + 15)

In [97]:
max_loc

(2114, 300)

In [115]:
grid_coord_dict[(0,0)]

(195, 35)

In [104]:
mouseclick(2114/2, 300/2)

In [ ]:
#DO SOMETHING (generate mouseclick on the board)
#retake picture, generate grid, do it again

In [ ]:
mystery_squares = np.where(grid == -1)
mystery_squares = zip(mystery_squares[0], mystery_squares[1])
n = random.randint(0, len(mystery_squares) - 1)
click_target_x = (max_loc[0] + grid_coord_dict[mystery_squares[n]][1])/2 + 15
click_target_y= (max_loc[1] + grid_coord_dict[mystery_squares[n]][0])/2 + 15

mouseclick(click_target_x,click_target_y)
mouseclick(click_target_x,click_target_y)

In [29]:
def generate_grid(grid_locations, picture):
    my_grid = np.zeros((9,9))
    ones = cv2.matchTemplate(picture, grid_one, cv2.TM_CCOEFF_NORMED)
    twos = cv2.matchTemplate(picture, grid_two, cv2.TM_CCOEFF_NORMED)
    threes = cv2.matchTemplate(picture, grid_three, cv2.TM_CCOEFF_NORMED)
    empties = cv2.matchTemplate(picture, empty, cv2.TM_CCOEFF_NORMED)
    mysteries = cv2.matchTemplate(picture, mystery, cv2.TM_CCOEFF_NORMED)
    adj = 2
    #there is a clear path to rid ourselves of this for loop. Implement later.
    for loc in grid_locations:
        coords = grid_loc_dict[loc]
        one_sim = ones[loc[0] + adj, loc[1] + adj]
        two_sim = twos[loc[0] + adj, loc[1] + adj]
        three_sim = threes[loc[0] + adj, loc[1] + adj]
        empty_sim = empties[loc[0] + adj, loc[1] + adj]
        mystery_sim = mysteries[loc[0], loc[1]]
        ind = np.argmax([empty_sim, one_sim, two_sim, three_sim, mystery_sim])
        my_grid[coords[0], coords[1]] = -1 if ind == 4 else ind
    return my_grid

In [26]:
mines = ImageGrab.grab(bbox = (max_loc[0]-490, max_loc[1]-85, (max_loc[0]) + 545, (max_loc[1]) + 1100)).convert('L')  # X1,Y1,X2,Y2
mines.show()
mines = np.array(mines)

IOError: cannot identify image file '/var/folders/fq/j4n3bx35559ctrhbm0cc_wg80000gn/T/pyscreenshot_screencapture_hgz51t.png'

In [30]:
grid = generate_grid(grid_locations, mines2)

IndexError: index 582 is out of bounds for axis 1 with size 555

In [ ]:
for i in range(10):
    mystery_squares = np.where(grid == -1)
    mystery_squares = zip(mystery_squares[0], mystery_squares[1])
    n = random.randint(0, len(mystery_squares) - 1)
    click_target_x = (max_loc[0] + grid_coord_dict[mystery_squares[n]][1])/2 + 15
    click_target_y= (max_loc[1] + grid_coord_dict[mystery_squares[n]][0])/2 + 15

    mouseclick(click_target_x, click_target_y)
    mouseclick(click_target_x, click_target_y)

    mines2 = ImageGrab.grab(bbox = (max_loc[0], max_loc[1], max_loc[0] + 610, max_loc[1] + 780)).convert('L')  # X1,Y1,X2,Y2
    #mines2.show()
    mines2 = np.array(mines2)
    grid = generate_grid(grid_locations, mines2)

In [26]:
def finalize_grid_locations():
    fullscreen = ImageGrab.grab().convert('L')  # X1,Y1,X2,Y2
    fs = np.array(fullscreen)
    simil_mat = cv2.matchTemplate(fs, minesweep, cv2.TM_CCOEFF_NORMED)
    thing = cv2.minMaxLoc(simil_mat)
    max_loc = thing[3]
    mines = ImageGrab.grab(bbox = (max_loc[0], max_loc[1], (max_loc[0]) + 610, (max_loc[1]) + 780)).convert('L')  # X1,Y1,X2,Y2
    #mines.show()
    mines = np.array(mines)
    mysteries = cv2.matchTemplate(mines, mystery, cv2.TM_CCOEFF_NORMED)
    grid_locations = remove_close_points(find_grid_locations(mysteries), mysteries)
    return max_loc, grid_locations

In [39]:
def play_minesweeper():
    max_loc, grid_locations = finalize_grid_locations()
    #create dictionary between locations and coordinates
    grid_loc_dict = dict()
    for i in range(len(grid_locations)):
        tup = grid_locations[i]
        grid_loc_dict[tup] = (i/9, i % 9)

    grid_coord_dict = dict()
    for x in grid_loc_dict.items():
        grid_coord_dict[x[1]] = x[0]
    grid = np.negative(np.ones((9,9)))
    
    for i in range(7):
        mystery_squares = np.where(grid == -1)
        mystery_squares = zip(mystery_squares[0], mystery_squares[1])
        n = random.randint(0, len(mystery_squares) - 1)
        click_target_x = (max_loc[0] + grid_coord_dict[mystery_squares[n]][1])/2 + 15
        click_target_y= (max_loc[1] + grid_coord_dict[mystery_squares[n]][0])/2 + 15

        mouseclick(click_target_x,click_target_y)
        mouseclick(click_target_x,click_target_y)
        mines2 = ImageGrab.grab(bbox = (max_loc[0], max_loc[1], max_loc[0] + 610, max_loc[1] + 780)).convert('L')  # X1,Y1,X2,Y2
        #mines2.show()
        mines2 = np.array(mines2)
        grid = generate_grid(grid_locations, mines2, grid_loc_dict)
        sleep(1)      

In [46]:
play_minesweeper()

In [ ]:
def alg(grid):
    mysery_count = count_adjacent(coord, -1)
    flag_count = count_adjacent()
    
    

In [ ]:
x = np.arange(9.).reshape(3, 3)

In [ ]:
x

In [ ]:
thing = np.where(x > 4)

In [ ]:
zip(thing[0], thing[1])

In [ ]:
plt.imshow(pic, cmap='gray')

In [ ]:
for x in all_my_lists: 
    if len(x) != 9:
        print(x)

In [ ]:
len(sum(all_my_lists,[]))

In [ ]:
length_arr = []  
for arr in all_my_lists:
    length_arr.append((max(zip(*arr)[1]) - min(zip(*arr)[1]))/(len(arr)-1))
length_arr     

In [ ]:
def binarize(matrix, threshold=0.7):
    temp = matrix
    for m in range(len(temp)):
        for n in range(len(temp[m])):
            if temp[m][n] < threshold:
                temp[m][n] = 0.0
            else:
                temp[m][n] = 1.0
    return temp

In [ ]:
bwminesweep = binarize(minesweep_1)
plt.imshow(bwminesweep, cmap='gray')

In [ ]:
fig, ax = plt.subplots()
ax.imshow(minesweep_1, interpolation='nearest', cmap=plt.cm.gray)

contours = find_contours(minesweep_1, 0.8)
c = 0
for n, contour in enumerate(contours):
    if c < 3000:
        if len(contour[:, 1]) > 60:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
        c += 1
    else:
        break
ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
contours

In [ ]:
enumerate(contours)

In [ ]:
type(im)

In [ ]:
im = im.convert('RGB')


In [ ]:
im.show(im)

In [ ]:
imarr = np.array(im)

In [ ]:
imarr.shape

In [ ]:
plt.imshow(temp, cmap='gray')

In [ ]:
plt.imshow(minesweep_1, cmap = 'gray')

In [ ]:
temp = ndi.imread('grid_square.png', mode='L')


In [ ]:
temp.shape

In [ ]:
info = cv2.minMaxLoc(thing)

In [ ]:
x1 = info[3][0]/2
x2 = x1 + 50
y1 = info[3][1]/2
y2 = y1 + 50

In [ ]:
if __name__ == '__main__':
    # part of the screen
    im2 = ImageGrab.grab(bbox=(x1, y1, x2, y2))  # X1,Y1,X2,Y2
    im2.show()